# Diabetic Retinopathy Detection - Jeffery A. Brown

# Dependencies

In [1]:
%matplotlib inline
# %matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D    # Support 3D graphing

import numpy as np
import pandas as pd
from pandas.plotting import table
import math
import random
import shutil
import os

# from imutils import paths
# import argparse
# import pickle
# import cv2

from pprint import pprint

# Visualization
import graphviz
import pydotplus
from IPython.display import Image

# Machine Learning - Data Preparation and Pre-Processing
from sklearn.model_selection import train_test_split # Split data into training and testing samples
from sklearn.model_selection import cross_val_score  # Score a model using k-fold or other cross validation

from sklearn.preprocessing import OneHotEncoder   # Convert categorical integer features (X) to One-Hot encoded values
from sklearn.preprocessing import LabelEncoder    # Convert categorical labeled values to categorical integer values
from sklearn.preprocessing import LabelBinarizer  # Convert categorical labeled values to Binary encoded values

from sklearn.preprocessing import StandardScaler  # Scale numerical features to standard normal distribution
from sklearn.preprocessing import MinMaxScaler    # Scale numerical values based upon mix/max values

# Machine Learning - Sci-Kit Learn - Models - Regression
from sklearn.linear_model import LinearRegression  # TBD
from sklearn.linear_model import Lasso             # TBD
from sklearn.linear_model import Ridge             # TBD
from sklearn.linear_model import ElasticNet        # TBD

# Machine Learning - Sci-Kit Learn - Models - Classification
from sklearn.linear_model import LogisticRegression   # Logistic Regression Classifier
from sklearn import tree                              # Decision Tree Classifier
from sklearn.ensemble import RandomForestClassifier   # Random Forest Classifier
from sklearn import svm                               # Support Vector Machine Classifier
from sklearn.neighbors import KNeighborsClassifier    # K-Nearest Neighbors (KNN)

# Machine Learning - GridSearch for Hyper-Parameter tuning
from sklearn.model_selection import GridSearchCV      # Grid Search

# Machine Learning - Quantify Model Performance
from sklearn.metrics import mean_squared_error    # Mean Squared Error (MSE) metric
from sklearn.metrics import r2_score              # R-squared (Coefficient of Determination) metric
from sklearn.metrics import confusion_matrix      # Generate a confusion matrix (actual vs. predicted counts)
from sklearn.metrics import classification_report # Calculate metrics for prediction performance
from sklearn.metrics import precision_score       # Calculate the precision: Tp / (Tp + Fp) => Ability to avoid false negatives
from sklearn.metrics import recall_score          # Calculate the recall: Tp / (Tp + Fn) => Ability to find all positive samples
from sklearn.metrics import f1_score              # Calculate the F1 score: 2*(precision*recall)/(precision+recall)

# Machine Learning - Dataset Generation
from sklearn.datasets import make_regression     # Generate linear data
from sklearn.datasets import make_s_curve        # Generate nonlinear data
from sklearn.datasets import make_blobs          # Generate blobs for classification
from sklearn.datasets import make_circles        # Generate circles for classification
from sklearn.datasets import load_iris           # Sample multi-class dataset for classification
from sklearn.datasets import make_classification # Generate datasets for classification

# Machine Learning - Keras (Tensorflow) - Models
from keras.models import Sequential               # Sequential model serving as foundation for neural network
# from keras.layers import Dense                    # Nodes for specifying input, hidden, and output layers
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense

# Machine Learning - Keras (Tensorflow) - Convolutional Neural Networks (Image Classification)
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K

# Machine Learning - Keras (Tensorflow) - Optimizers
from keras.optimizers import SGD
from keras.optimizers import Adam

# Machine Learning - Keras (Tensorflow) - Encoding
from keras.utils import to_categorical            # One-Hot Encoder provided through Keras

# Machine Learning - Keras (Tensorflow) - Other related Tools
from keras.utils import plot_model                # Plot a neural network model
from keras.models import load_model               # Load a saved machine learning model
from keras.preprocessing import image             # Loads an image for application of machine learning
from keras.preprocessing.image import img_to_array # Converts an image to a numpy array
from keras.preprocessing.image import ImageDataGenerator

# Machine Learning - Keras (Tensorflow) -  Dataset Generation
from keras.datasets import mnist                  # Images: Handwritten digits 0-9 (28x28 grayscale, 60K train, 10K test)


Using TensorFlow backend.


# Predict the Class for Official Test Images

## Load the Saved Convolutional Neural Network Model

In [2]:
# Load a saved model
model = load_model("./saved_model.h5")

W0907 18:40:45.327334 21972 deprecation_wrapper.py:119] From C:\Users\Jeff\AppData\Local\conda\conda\envs\PythonTF\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0907 18:40:45.355404 21972 deprecation_wrapper.py:119] From C:\Users\Jeff\AppData\Local\conda\conda\envs\PythonTF\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0907 18:40:45.399335 21972 deprecation_wrapper.py:119] From C:\Users\Jeff\AppData\Local\conda\conda\envs\PythonTF\lib\site-packages\keras\backend\tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0907 18:40:45.400335 21972 deprecation_wrapper.py:119] From C:\Users\Jeff\AppData\Local\conda\conda\envs\PythonTF\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is depre

## Copy the Official Test Data to a Structure Compliant with ImageDataGenerator flow_from_directory function

In [12]:
# Function to copy the official test data into a folder structure that
# can be used with ImageDataGenerator and the flow_from_directory method,
# which requires the test images to be in a *subfolder* under the main test data folder!
# So unfortunately, since cannot change the test data folder structure I need to *copy*
# all of the test data to another folder with proper structure
def copy_official_test_images( a_source_dir = None, a_dest_dir = None ):
    
    # Basic argument checking
    if (a_source_dir is None) or (a_dest_dir is None):
        print(f"Error: Source or Destination directory not specified")
        return False
            
    # Create the destination folder if it doesn't already exist
    if not os.path.exists(a_dest_dir):
            os.mkdir(a_dest_dir)

    # Create the required subfolder in the destination folder if it doesn't already exist
    dir_subfolder = os.path.join(a_dest_dir, "test_subfolder")
    if not os.path.exists(dir_subfolder):
            os.mkdir(dir_subfolder)

    # Copy all images in the a_src_folder to the newly created dir_subfolder
    src_files = os.listdir(a_source_dir)
    
    for file_name in src_files:
        
        full_file_name = os.path.join(a_source_dir, file_name)
        
        if os.path.isfile(full_file_name):
            print(f"Copying image {full_file_name} to {dir_subfolder}... ", end='')
            try:
                shutil.copy2(full_file_name, dir_subfolder)
                print(f"done.")

            except FileNotFoundError:
                print(f"error while attempting to copy.")

In [13]:
# data/test_images - Source of official testing images in a single folder (used only to submit official results)
# dir_src_test = "../input/aptos2019-blindness-detection/test_images"
dir_src_test = "./data/test_images"
dir_official_test = "./official_test"

In [14]:
copy_official_test_images( dir_src_test, dir_official_test )

Copying image ./data/test_images\0005cfc8afb6.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\003f0afdcd15.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\006efc72b638.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\00836aaacf06.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\009245722fa4.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\009c019a7309.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\010d915e229a.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\0111b949947e.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\01499815e469.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\0167076e7089.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\01c31b10ab99.png to ./official_test\t

Copying image ./data/test_images\0ed192cd67a2.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\0edc0c33e929.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\0ee72c039036.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\0ef1f5075178.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\0f1613975335.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\0f1a967313c3.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\0f23c3028206.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\0f61878bfd0e.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\0f8fc606e17c.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\0f96e3e4a5a0.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\0fa8a299f431.png to ./official_test\t

Copying image ./data/test_images\1d7c4c12769e.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\1de193623108.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\1e0309115a25.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\1e04a80f12dd.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\1e15a8fa476f.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\1e34e33a0988.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\1ed74b13dc38.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\1f2018b62574.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\1f244c85e046.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\1fb1acde9818.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\1fb4db8834df.png to ./official_test\t

Copying image ./data/test_images\2f9a5d2c4ee5.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\2fb539602f57.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\2febc332f628.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\2febce215663.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\2ffd44200e6f.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\3006fdc07d57.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\30376d8cde4e.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\3040ae4067b0.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\305635aaec8a.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\308924f570d7.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\3099467aeef2.png to ./official_test\t

Copying image ./data/test_images\3ef3056cc3d0.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\3f007a2060c0.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\3f03980754e7.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\3f3e750580d4.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\3f42e760cc3e.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\3f5420546cba.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\3fa9277a3780.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\3fd59954189f.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\3fdbdbc04be6.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\401279eb879a.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\408ea007f294.png to ./official_test\t

...

Copying image ./data/test_images\bc7419fa4437.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\bc8d12d09cf0.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\bc9d8ac89c82.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\bca38b014cdd.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\bccb2c2c4216.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\bd027726e13b.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\bd3bd6aac8e2.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\bda2d5ca6f7c.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\bde27560a3b8.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\be110f3aad85.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\be640164686b.png to ./official_test\t

Copying image ./data/test_images\cad7be9b1e4a.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\caf8c68b4b7f.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\cb004a78aecf.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\cb00fbc43f3a.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\cb2319096cee.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\cb2eaa6e87ad.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\cb484b214c95.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\cba1fb3d80e0.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\cbdc19b70634.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\cbde1c64f27e.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\cbf8916e5abe.png to ./official_test\t

Copying image ./data/test_images\d9b846ab7972.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\d9c9e94df371.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\da4e9278ccfe.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\da58ea1e0407.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\da68a7779f0d.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\da763f0c1a0f.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\da807f93050c.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\da91be7eeb4f.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\daa2d1d05f49.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\dabe64a3b18b.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\dadc953e79e2.png to ./official_test\t

Copying image ./data/test_images\ebb40c984d56.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\ebb70180b81a.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\ec1ec61a3354.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\ec248c3ecd10.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\ec353d42359d.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\ec4309136788.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\ec5b8792b1a2.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\ec71f67e2cd1.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\ec727873cdb7.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\ec8f3ec97245.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\ed0ccec62a3a.png to ./official_test\t

Copying image ./data/test_images\f6578f3dfafe.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\f68ada6a0892.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\f6a79dbb4339.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\f6a9c937e6d6.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\f7113d580492.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\f72a18c76408.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\f73e8ec73179.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\f756b1c4c52b.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\f75b65c8cc4c.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\f785337edf07.png to ./official_test\test_subfolder... done.
Copying image ./data/test_images\f7b4b4cf9161.png to ./official_test\t

In [23]:
# Count how many test images are populated
# test_tally_file = "../input/aptos2019-blindness-detection/test.csv"
test_tally_file = "./data/test.csv"

# Load test counts into a dataframe
test_tally_df = pd.read_csv(test_tally_file)
len(test_tally_df)

1928

## Configure the Image Generator for the Official Test Images

In [25]:
# image_dims = (2136, 3216, 3)   # Height, Width, Depth (RGB = 3, Grayscale = 1)
image_dims = (96, 96, 3)         # Height, Width, Depth (RGB = 3, Grayscale = 1)

# Create a Image Data Generator for the official testing data
official_test_datagen = ImageDataGenerator()

# Define generator batch size and step size
official_test_n_images = len(test_tally_df)
official_test_n_batch_size = 1
official_test_n_step_size = official_test_n_images // official_test_n_batch_size

# Associate the generator with the training images
official_test_generator = official_test_datagen.flow_from_directory(
    directory=dir_official_test,             # Folder containing a subfolder with official training images
    target_size=(image_dims[1], image_dims[0]),       # (Width, Height)
    color_mode="rgb",               # Color images
    batch_size=official_test_n_batch_size,   # Batch size
    class_mode=None,                # Number of classes > 2, so using "categorical" vs "binary"
    shuffle=False,                   # Shuffle the training samples randomly
    seed=1                          # Random number seed to use for shuffle
)

Found 1928 images belonging to 1 classes.


## Predict the Classifications based upon the Images

In [26]:
# Predict the output (using the test_generator)
official_test_generator.reset()
official_pred_onehot = model.predict_generator(official_test_generator,
                             steps=len(official_test_generator),
                             verbose=1)

official_predicted_class = np.argmax(official_pred_onehot, axis=1)
official_predicted_class

1928/1928 [==============================] - 74s 38ms/step


array([2, 3, 0, ..., 3, 0, 2], dtype=int64)

## Save the Predictions from the Official Test Data

In [27]:
# Save the prediction results to a file

# Get the list of test image id codes used by the test generator to make the predictions
official_test_id_code_list = [ (x.split("\\"))[1].replace(".png","") for x in official_test_generator.filenames ]

# Associate the test image id codes and the preoductions in a dataframe
official_results_df = pd.DataFrame({"id_code": official_test_id_code_list, "diagnosis": official_predicted_class})
official_results_df.to_csv("./submission.csv",index=False)
official_results_df

,id_code,diagnosis
0,0005cfc8afb6,2
1,003f0afdcd15,3
2,006efc72b638,0
3,00836aaacf06,2
4,009245722fa4,2
...,...,...
1923,ff2fd94448de,0
1924,ff4c945d9b17,0
1925,ff64897ac0d8,3
1926,ffa73465b705,0
